![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)
![SSU AILAB Logo](http://ailab.ssu.ac.kr/rb/layouts/default/image/common/logo.png)

* ### Spark Context import 확인
(설치 가이드대로 설치했을 경우, sc 라는 변수의 이름으로 SparkContext 가 자동으로 생성되어 있음)

In [1]:
type(sc)

pyspark.context.SparkContext

In [2]:
sc.__dict__

{'_accumulatorServer': <pyspark.accumulators.AccumulatorServer at 0x110525748>,
 '_batchSize': 0,
 '_callsite': CallSite(function='<module>', file='/Users/ai/anaconda3/lib/python3.6/site-packages/IPython/utils/py3compat.py', linenum=186),
 '_conf': <pyspark.conf.SparkConf at 0x1105256d8>,
 '_encryption_enabled': False,
 '_javaAccumulator': JavaObject id=o17,
 '_jsc': JavaObject id=o14,
 '_pickled_broadcast_vars': <pyspark.broadcast.BroadcastPickleRegistry at 0x11006cb28>,
 '_python_includes': [],
 '_temp_dir': '/private/var/folders/_p/rfs0fb9n7b52r9m1tbwndftr0000gn/T/spark-823153ef-c50a-4f3c-94bd-c20665e2a72d/pyspark-20d6e879-947d-497e-8d0b-5156dc44cb95',
 '_unbatched_serializer': PickleSerializer(),
 'appName': 'PySparkShell',
 'environment': {'PYTHONHASHSEED': '0'},
 'master': 'local[2]',
 'profiler_collector': None,
 'pythonExec': 'python',
 'pythonVer': '3.6',
 'serializer': AutoBatchedSerializer(PickleSerializer()),
 'sparkHome': None}

1) #### 사용자의 Spark Application 이름 바꾸기

In [3]:
sc.appName = "TestApp"

In [4]:
sc.appName

'TestApp'

* ### 파일로 부터 Spark RDD 생성
(4개의 파티션 생성)

In [5]:
data = sc.textFile("sample.txt", 4)

* ### 파티션별 데이터 확인

In [6]:
data.glom().collect()

[['Once more',
  'I summon you',
  'Out of the past',
  'With poignant love,',
  'You who nourished the poet'],
 ['And the lover.', 'I see your gary eyes', 'Looking out to sea'],
 ['In those Rockport summers,', 'Keeping a distance', 'Within the closeness'],
 ['Which was never intrusive', 'Opening out', 'Into the world.']]

* ### 파티션의 개수를 6개로 변경후 확인

In [7]:
data1 = data.repartition(6)

In [8]:
data1.glom().collect()

[[],
 ['Once more',
  'I summon you',
  'Out of the past',
  'With poignant love,',
  'You who nourished the poet'],
 ['Which was never intrusive', 'Opening out', 'Into the world.'],
 [],
 ['In those Rockport summers,', 'Keeping a distance', 'Within the closeness'],
 ['And the lover.', 'I see your gary eyes', 'Looking out to sea']]

In [9]:
print(data1.getNumPartitions())

6


* ### xrange, range 차이점 (python 내용)
(xrange 는 lazy evaluation)

In [10]:
# dataXrange = xrange(1,10)

In [11]:
# type(dataXrange)

In [12]:
dataRange = range(1,10)

In [13]:
type(dataRange)

range

In [14]:
# dataXrange

In [15]:
dataRange

range(1, 10)

* ### 객체로 부터 RDD 생성

In [16]:
# xrangeRDD = sc.parallelize(dataXrange)

In [17]:
rangeRDD = sc.parallelize(dataRange,8)

* ### toDebugString 를 통한 RDD 의 lineage 확인

In [18]:
# xrangeRDD.toDebugString()

In [19]:
rangeRDD.toDebugString()

b'(8) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |  ParallelCollectionRDD[9] at parallelize at PythonRDD.scala:195 []'

* ### xrange, range 타입에 따라 생성되는 RDD 의 타입 확인

In [20]:
# print 'type of RangeRDD: {0}'.format(type(xrangeRDD))

In [21]:
print('type of XrangeRDD: {0}'.format(type(rangeRDD)))

type of XrangeRDD: <class 'pyspark.rdd.PipelinedRDD'>


In [22]:
print('id : {0}'.format(rangeRDD.id()))
# print('id : {0}'.format(xrangeRDD.id()))

id : 10


* ### function 을 정의하고 Spark 의 map 함수 적용

In [49]:
def sub(value):
    return (value - 1)

subRDD = rangeRDD.map(sub)

print("원본 데이터 : ", rangeRDD.glom().collect())
print("sub 데이터 : ",subRDD.glom().collect())


원본 데이터 :  [[1], [2], [3], [4], [5], [6], [7], [8, 9]]
sub 데이터 :  [[0], [1], [2], [3], [4], [5], [6], [7, 8]]


* ### Map 함수와 flatMap 차이점

In [50]:
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList,4)

sigularAndPluralWordsRDDMap = wordsRDD.map(lambda x: (x, x + 's'))
sigularAndPluralWordsRDD = wordsRDD.flatMap(lambda x: (x, x + 's'))

print(sigularAndPluralWordsRDDMap.collect())
print(sigularAndPluralWordsRDD.collect())
print(sigularAndPluralWordsRDDMap.count())
print(sigularAndPluralWordsRDD.count())

[('cat', 'cats'), ('elephant', 'elephants'), ('rat', 'rats'), ('rat', 'rats'), ('cat', 'cats')]
['cat', 'cats', 'elephant', 'elephants', 'rat', 'rats', 'rat', 'rats', 'cat', 'cats']
5
10


In [53]:
simpleRDD = sc.parallelize([2,3,4])
print(simpleRDD.map(lambda x: range(1, x)).collect())

[range(1, 2), range(1, 3), range(1, 4)]


In [51]:
simpleRDD = sc.parallelize([2,3,4])
print(simpleRDD.map(lambda x: list(range(1, x))).collect())#python 2 결과 - [[1],[1,2],[1,2,3]]
print(simpleRDD.flatMap(lambda x: range(1, x)).collect())


[[1], [1, 2], [1, 2, 3]]
[1, 1, 2, 1, 2, 3]


* ### Spark의 filter 함수 적용 과 lambda 사용법

In [54]:
def five(value):
    if(value < 5):
        return True
    else:
        return False

filteredRDD = subRDD.filter(five)
filteredRDD2 = subRDD.filter(lambda x: x > 5)

print("sub 데이터 : ", subRDD.glom().collect())
print("Partition별 5보다 작은 데이터", filteredRDD.glom().collect())
print("Partition별 5보다 큰 데이터", filteredRDD2.glom().collect())
print("filteredRDD Partition 개수 : ", filteredRDD.getNumPartitions())
print("filteredRDD2 Partition 개수 : ", filteredRDD2.getNumPartitions())

sub 데이터 :  [[0], [1], [2], [3], [4], [5], [6], [7, 8]]
Partition별 5보다 작은 데이터 [[0], [1], [2], [3], [4], [], [], []]
Partition별 5보다 큰 데이터 [[], [], [], [], [], [], [6], [7, 8]]
filteredRDD Partition 개수 :  8
filteredRDD2 Partition 개수 :  8


* ### takeSample 함수를 사용하여 샘플 데이터 추출
(withReplacement 는 중복 허용 여부, num 는 샘플 개수, seed 를 주면 항상 같은 샘플 추출)

In [55]:
print(filteredRDD.takeSample(withReplacement=True, num=6))
print(filteredRDD.takeSample(withReplacement=False, num=6))

[0, 0, 3, 4, 4, 0]
[0, 4, 1, 2, 3]


In [56]:
print(filteredRDD.takeSample(withReplacement=True, num=6, seed=500))
print(filteredRDD.takeSample(withReplacement=True, num=6, seed=500))

[1, 3, 2, 2, 0, 4]
[1, 3, 2, 2, 0, 4]


In [57]:
print(filteredRDD.takeSample(withReplacement=True, num=6))
print(filteredRDD.takeSample(withReplacement=True, num=6))

[1, 4, 0, 1, 1, 4]
[0, 3, 4, 1, 3, 4]


* ### reduceByKey, groupByKey
(각각의 return type 숙지, groupByKey 사용법 등 주의)

![reduceByKey() figure](http://spark-mooc.github.io/web-assets/images/reduce_by.png)
![groupByKey() figure](http://spark-mooc.github.io/web-assets/images/group_by.png)

#### 1) GroupByKey 에 관한 내용

In [58]:
s = sc.parallelize([('kim', 'ee'), ('choi', 'cs'), 
                    ('lee', 'cs'), ('park', 'ee'), ('park', 'biology')])
sGrouped = s.groupByKey().collect()
print("student : ", s)
for key, value in sGrouped:
    print('{0} : {1}'.format(key, value))
print('\n')
for key, value in sGrouped:
    print('{0} : {1}'.format(key, list(value)))

student :  ParallelCollectionRDD[233] at parallelize at PythonRDD.scala:195
choi : <pyspark.resultiterable.ResultIterable object at 0x11062dc18>
lee : <pyspark.resultiterable.ResultIterable object at 0x11062d908>
kim : <pyspark.resultiterable.ResultIterable object at 0x11062d208>
park : <pyspark.resultiterable.ResultIterable object at 0x11062d1d0>


choi : ['cs']
lee : ['cs']
kim : ['ee']
park : ['ee', 'biology']


In [30]:
#groupByKey 1
s = sc.parallelize([('kim', 'ee'), ('choi', 'cs'), ('lee', 'cs'), ('park', 'ee')])
sGrouped = s.groupByKey().collect()
print("student : ", s)
for key, value in sGrouped:
    print('{0} : {1}'.format(key, value))
print('\n')
for key, value in sGrouped:
    print('{0} : {1}'.format(key, list(value)))

student :  ParallelCollectionRDD[36] at parallelize at PythonRDD.scala:195
choi : <pyspark.resultiterable.ResultIterable object at 0x1106122b0>
lee : <pyspark.resultiterable.ResultIterable object at 0x110612668>
kim : <pyspark.resultiterable.ResultIterable object at 0x110612358>
park : <pyspark.resultiterable.ResultIterable object at 0x110612908>


choi : ['cs']
lee : ['cs']
kim : ['ee']
park : ['ee']


In [31]:
#groupByKey 2
word = sc.parallelize([('am', 4), ('boy', 2), ('am', 2), ('boy', 3)])
wordGroup = word.groupByKey()
for key, value in wordGroup.collect():
    print(key, list(value))

am [4, 2]
boy [2, 3]


In [32]:
#groupByKey sum2
word = sc.parallelize([('am', 4), ('boy', 2), ('am', 2), ('boy', 3)])
wordGroup = word.groupByKey()
for key, value in wordGroup.collect():
    print(key, sum(value))

am 6
boy 5


In [59]:
word = sc.parallelize([('am', 4), ('boy', 2), ('am', 2), ('boy', 3)])
wordGroup = word.groupByKey()
wc = wordGroup.map(lambda (k, v) : (k, sum(v)))
for key, value in wc.collect():
    print(key, value)

SyntaxError: invalid syntax (<ipython-input-59-7ddda73dd1e8>, line 3)

In [60]:
#groupByKey map3 
word = sc.parallelize([('am', 4), ('boy', 2), ('am', 2), ('boy', 3)])
wordGroup = word.groupByKey()
wc = wordGroup.map(lambda kv : (kv[0], sum(kv[1])))
for key, value in wc.collect():
    print(key, value)

am 6
boy 5


In [61]:
#groupByKey mapValues3
word = sc.parallelize([('am', 4), ('boy', 2), ('am', 2), ('boy', 3)])
wordGroup = word.groupByKey()
wc = wordGroup.mapValues(sum)
for key, value in wc.collect():
    print(key, value)

am 6
boy 5


In [62]:
#groupByKey mapValues4
word = sc.parallelize([('am', 4), ('boy', 2), ('am', 2), ('boy', 3)])
wordGroup = word.groupByKey()
wc = wordGroup.mapValues(list)
for key, value in wc.collect():
    print(key, value)

am [4, 2]
boy [2, 3]


In [64]:
word = sc.parallelize([('am', 4), ('boy', 2), ('am', 2), ('boy', 3)])
wordGroup = word.groupByKey()
wc = wordGroup.mapValues(lambda x : [i**2 for i in list(x)])
for key, value in wc.collect():
    print(key, value)

am [16, 4]
boy [4, 9]


#### 2) reduceByKey 에 관한 내용

In [66]:
#reduceByKey
word = sc.parallelize([('am', 4), ('boy', 2), ('am', 2),
                       ('am', 3),('boy', 3),('boy', 5),('boy', 4)])
wc = word.reduceByKey(lambda a, b : a + b)
for key, value in wc.collect():
    print(key, value)

am 9
boy 14


In [37]:
#reduceBykey
def add(a, b):
    return a + b

word = sc.parallelize([('am', 4), ('boy', 2), ('am', 2), ('boy', 3)])
wc = word.reduceByKey(add)
for key, value in wc.collect():
    print(key, value)

am 6
boy 5


In [67]:
# python map
line = "this is a test"
print(line.split())
sp = map(lambda a : (a, 1), line.split())
print(sp)
for x in sp:
    print(x)

['this', 'is', 'a', 'test']
('this', 1)
('is', 1)
('a', 1)
('test', 1)


In [2]:
text = sc.textFile("short.txt")
text.collect()

['this is a test', 'this is the second test', 'this may be the thing']

In [3]:
#pyspark map
text = sc.textFile("short.txt")
wc = text.map(lambda line : line.split())
wc.collect()

[['this', 'is', 'a', 'test'],
 ['this', 'is', 'the', 'second', 'test'],
 ['this', 'may', 'be', 'the', 'thing']]

In [4]:
#pyspark flatMap
text = sc.textFile("short.txt")
wc = text.flatMap(lambda line : line.split())
wc.collect()

['this',
 'is',
 'a',
 'test',
 'this',
 'is',
 'the',
 'second',
 'test',
 'this',
 'may',
 'be',
 'the',
 'thing']

In [41]:
#pyspark flatMap reduceByKey
text = sc.textFile("short.txt")
wc = text.flatMap(lambda line : line.split()).map(lambda word : (word,1))
print(wc.collect())

[('this', 1), ('is', 1), ('a', 1), ('test', 1), ('this', 1), ('is', 1), ('the', 1), ('second', 1), ('test', 1), ('this', 1), ('may', 1), ('be', 1), ('the', 1), ('thing', 1)]


In [5]:
#pyspark flatMap reduceByKey
text = sc.textFile("short.txt")
wc = text.flatMap(lambda line : line.split()).map(lambda word : (word,1)) \
        .reduceByKey(lambda c1, c2: c1 + c2)
print(wc.collect())

[('this', 3), ('is', 2), ('test', 2), ('may', 1), ('a', 1), ('the', 2), ('second', 1), ('be', 1), ('thing', 1)]


In [6]:
text = sc.textFile("short.txt")
wc = text.flatMap(lambda line : line.split()).map(lambda word : (word,1))\
        .reduceByKey(lambda c1, c2: c1 + c2)
    
print(wc.sortByKey(ascending=True).collect())
print(wc.map(lambda x: (x[1], x[0])).sortByKey(ascending=False).collect())

[('a', 1), ('be', 1), ('is', 2), ('may', 1), ('second', 1), ('test', 2), ('the', 2), ('thing', 1), ('this', 3)]
[(3, 'this'), (2, 'is'), (2, 'test'), (2, 'the'), (1, 'may'), (1, 'a'), (1, 'second'), (1, 'be'), (1, 'thing')]


* ### RDD의 reduce 함수를 적용(substract 를 사용시 주의)
reduce 는 function 이 associative 하고 commutative 해야한다.

In [44]:
print("[p8]data : ", filteredRDD.glom().collect())
# - 는 associative 하지만, commutative하지 못하다.
print("[p8]reduce(sum) : ", filteredRDD.reduce(lambda a, b: a + b))
print("[p8]reduce(sub) : ", filteredRDD.reduce(lambda a, b: a - b))

# partition에 따라 sub 함수의 값이 달라짐(commutative하지 못하기 때문)
rep_filteredRDD = filteredRDD.repartition(3)
print("[p3]data : ", rep_filteredRDD.glom().collect())

# sum 은 partition 을 나눠도 associative 하고 commutative 하기 때문에 결과 같음
print("[p3]reduce(sum) : ", filteredRDD.repartition(3).reduce(lambda a, b: a + b))
print("[p3]reduce(sub) : ", filteredRDD.repartition(3).reduce(lambda a, b: a - b))

[p8]data :  [[0], [1], [2], [3], [4], [], [], []]
[p8]reduce(sum) :  10
[p8]reduce(sub) :  -10
[p3]data :  [[], [0, 2], [1, 3, 4]]
[p3]reduce(sum) :  10
[p3]reduce(sub) :  4


In [45]:
pairRDD = sc.parallelize([('a',1),('a',2),('b',1)])
#print(pairRDD.groupByKey().map(lambda (k,v): (k, sum(v))).collect())
print(pairRDD.groupByKey().mapValues(lambda x: list(x)).collect())
print(pairRDD.groupByKey().map(lambda x: (x[0], sum(x[1]))).collect())
print(pairRDD.groupByKey().mapValues(lambda x: sum(x)).collect())
print(pairRDD.reduceByKey(add).collect())

[('b', [1]), ('a', [1, 2])]
[('b', 1), ('a', 3)]
[('b', 1), ('a', 3)]
[('b', 1), ('a', 3)]


* ###  WordCount 방법 비교

In [7]:
repetitiveRDD = sc.parallelize(["i","am", "boy", "i"])

text1 = repetitiveRDD \
.flatMap(lambda line: line.split()).map(lambda word: (word, 1)) \
.reduceByKey(lambda v1, v2: v1 + v2).collect() # collectAsMap() 사용 시 dictionary 형태로 return

print("reduceByKey : ", text1)

text2 = repetitiveRDD \
.flatMap(lambda line: line.split()).map(lambda word: (word, 1)) \
.groupByKey() \
.map(lambda kv: (kv[0], sum(kv[1]))).collect()
print("groupByKey : ", text2)

reduceByKey :  [('i', 2), ('am', 1), ('boy', 1)]
groupByKey :  [('i', 2), ('am', 1), ('boy', 1)]


In [47]:
repetitiveRDD = sc.parallelize(["i","am", "boy", "i"])

text1 = repetitiveRDD \
.flatMap(lambda line: line.split()).map(lambda word: (word, 1)) \
.reduceByKey(lambda v1, v2: v1 + v2)

print(text1.sortByKey(ascending=True).collect())
print(text1.map(lambda kv : (kv[1], kv[0])).sortByKey(ascending=False) \
      .collect())

[('am', 1), ('boy', 1), ('i', 2)]
[(2, 'i'), (1, 'am'), (1, 'boy')]


* ### RDD의 first 함수는 첫번 째 partition 의 첫번 째 데이터를 Return, take(1)은 first() 와 동일, takeOrdered 는 오름차순 정렬 후 Return

In [8]:
text = sc.textFile("sample.txt", 4) \
.flatMap(lambda line: line.split()).map(lambda word: (word, 1)) \
.reduceByKey(lambda v1, v2: v1+v2)
print(text.glom().collect())
print("\n== first ==")
print(text.first()) # First Partition의 첫번째 Element.
print("\n== take ==")
print(text.take(12))
print("\n== takeOrdered ==")
print(text.takeOrdered(3)) # Collect Action 후에 작은 Element 순으로.
print("\n== top ==")
 #Collect Action 후에 큰 Element 순으로.
print("based of key : ", text.top(5))
print("based of value : ", text.top(5, key = (lambda x : x[1])))

[[('Out', 1), ('of', 1), ('poignant', 1), ('poet', 1), ('And', 1), ('lover.', 1), ('eyes', 1), ('out', 2), ('sea', 1), ('Rockport', 1), ('Within', 1), ('Which', 1), ('was', 1)], [('you', 1), ('the', 5), ('With', 1), ('love,', 1), ('You', 1), ('who', 1), ('to', 1), ('Keeping', 1), ('a', 1), ('Opening', 1), ('Into', 1)], [('Once', 1), ('more', 1), ('summon', 1), ('past', 1), ('gary', 1), ('distance', 1), ('closeness', 1), ('never', 1), ('intrusive', 1)], [('I', 2), ('nourished', 1), ('see', 1), ('your', 1), ('Looking', 1), ('In', 1), ('those', 1), ('summers,', 1), ('world.', 1)]]

== first ==
('Out', 1)

== take ==
[('Out', 1), ('of', 1), ('poignant', 1), ('poet', 1), ('And', 1), ('lover.', 1), ('eyes', 1), ('out', 2), ('sea', 1), ('Rockport', 1), ('Within', 1), ('Which', 1)]

== takeOrdered ==
[('And', 1), ('I', 2), ('In', 1)]

== top ==
based of key :  [('your', 1), ('you', 1), ('world.', 1), ('who', 1), ('was', 1)]
based of value :  [('the', 5), ('out', 2), ('I', 2), ('Out', 1), ('of'